# VBA Automation 

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
import string
from tqdm import tqdm
import time
import sys
import os
import matplotlib.pyplot as plt

from scipy.stats.mstats import winsorize
import seaborn as sns

The cell below needs to be run as a preparation for the computation of the emissions.

In [ ]:
app = xw.App(visible=False)

opgee_file = xw.books.open('C:/Users/jing/Documents/Bocconi/DSBA/OPGEE/OPGEE_3.0b_BETA.xlsm') #Insert the path to the OPGEE VBA file
inputs_file = pd.read_excel('C:/Users/jing/Documents/Bocconi/DSBA/OPGEE/field_data_for_opgee.xlsx') #Insert the path to the inputs file
all_fields = inputs_file[inputs_file['production'] != 0].reset_index(drop = True) #Filter out fields with null production

all_letters = [letter for letter in string.ascii_uppercase]
double_letters = []
for first_letter in tqdm(all_letters):
    for second_letter in all_letters:
        double_letters.append(first_letter + second_letter)
all_letters += double_letters
all_letters = all_letters[all_letters.index('I'):all_letters.index('SM')+1]

The code below allows to take the fields inputs from a separate file, and to put them in the VBA OPGEE file to then run it. 
Given that it is possible to run the VBA code with at most 499 fields, it is possible to set two variables:
- n_fields: the number of fields to be filled in.
- n_iterations: the number of times the VBA file is run.

For instance, if n_fields = 499, and n_iterations = 3, then overall the emissions of 499x3 fields are computed.
For each iteration, a new file is generated, which contains the fields characteristics with their respective computed emissions. The file name is OPGEE_Data_n.csv, where n is the index of the file. 
The code automatically detects how many emissions have already been computed, and it starts from the appropriate field. For instance, if 3 files have already been generated with 20 fields each, then the code will start from the 61th field. 

In [ ]:
n_iterations = 1
n_fields = 499
running_times = []

for j in tqdm(range(n_iterations)):
    all_files = os.listdir()
    already_opgee = pd.DataFrame()
    last_index = 0

    for i in range(len(all_files)):
        if all_files[i][:10] == 'OPGEE_Data':
            last_index += 1
            already_opgee = pd.concat([already_opgee, pd.read_csv(all_files[i])])

    shift = already_opgee.shape[0]
    
    if (all_fields.shape[0] - shift - n_fields)//499 <= 0:
        n_fields = (all_fields.shape[0] - shift - n_fields) % 499
    
    x_data = all_fields.loc[shift : shift + n_fields - 1, ['country_name', 'field_name', 'field_age', 'field_depth', 'production', 'offshore', 'api', 'h2s']]
    x_data['Emission'] = 0

    opgee_file = xw.books.open('C:/Users/jing/Documents/Bocconi/DSBA/OPGEE/OPGEE_3.0b_BETA.xlsm') #Insert the path to the OPGEE VBA file
    inputs_file = pd.read_excel('C:/Users/jing/Documents/Bocconi/DSBA/OPGEE/field_data_for_opgee.xlsx') #Insert the path to the inputs file
    inputs_sheet = opgee_file.sheets('Inputs')
    results_sheet = opgee_file.sheets('Results')
    production_methods_default = inputs_sheet.range('G9:G17').value

    for i in range(n_fields):
        col = all_letters[i]
        inputs_sheet.range(f'{col}9:{col}17').value = production_methods_default
        inputs_sheet.range(f'{col}20').value = all_fields.loc[i + shift, 'country_name']
        inputs_sheet.range(f'{col}21').value = all_fields.loc[i + shift, 'field_name']
        inputs_sheet.range(f'{col}22').value = all_fields.loc[i + shift, 'field_age']
        inputs_sheet.range(f'{col}23').value = all_fields.loc[i + shift, 'field_depth']
        inputs_sheet.range(f'{col}24').value = all_fields.loc[i + shift, 'production']
        inputs_sheet.range(f'{col}31').value = all_fields.loc[i + shift, 'offshore']
        inputs_sheet.range(f'{col}34').value = all_fields.loc[i + shift, 'api']
        inputs_sheet.range(f'{col}42').value = all_fields.loc[i + shift, 'h2s']

    inputs_sheet.range('C2').value = n_fields + 1

    start_time = time.time()
    trial_macro = opgee_file.macro('BulkAssessment.UltraBulk_assessment')
    trial_macro()
    end_time = time.time()
    running_times.append((end_time - start_time)/60)

    end_bound = all_letters[n_fields-1]
    emissions = results_sheet.range(f'I193:{end_bound}193').value
    x_data['Emission'] = emissions
    x_data.to_csv(f'OPGEE_Data_{last_index}.csv')

    opgee_file.close()

The cell below needs to be run once all the emissions have been computed and all the files have been generated. 
It simply concatenates all the files to then generate a single file named OPGEE_Data.csv.

In [ ]:
all_files = os.listdir()
opgee = pd.DataFrame()

for i in range(len(all_files)):
    if all_files[i][:10] == 'OPGEE_Data':
        opgee = pd.concat([opgee, pd.read_csv(all_files[i])])

opgee = opgee.drop('Unnamed: 0', axis = 1)
opgee.to_csv('OPGEE_Data.csv')